In [6]:
!pip install mlxtend


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from google.colab import files
import pandas as pd

# Upload both orders.csv and treez.csv.csv
uploaded = files.upload()

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Load data
orders = pd.read_csv("order.csv")
trees = pd.read_csv("treez.csv.csv", low_memory=False)

print("Orders preview:")
print(orders.head())
print("\nTrees preview:")
print(trees.head())

In [ ]:
import re

def normalize_name(s):
    if pd.isna(s): return ""
    return re.sub(r"[^\w\s]", "", str(s).lower().strip())

kaggle_common_col = [c for c in trees.columns if "common" in c.lower()][0]

trees["common_norm"] = trees[kaggle_common_col].apply(normalize_name)
orders["common_norm"] = orders["COMMON_NAME"].apply(normalize_name)

valid_names = set(trees["common_norm"])
orders = orders[orders["common_norm"].isin(valid_names)]

print("✅ Filtered orders:", len(orders))
print(orders.head())

In [ ]:
transactions = orders.groupby("ORDER_ID")["common_norm"].apply(list).tolist()
print("Sample transaction baskets:")
for t in transactions[:5]:
    print(t)

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules


te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_enc = pd.DataFrame(te_ary, columns=te.columns_)

# Apriori
frequent_itemsets = apriori(df_enc, min_support=0.2, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

strong_rules = rules[(rules["confidence"] >= 0.7) & (rules["lift"] > 1)]
print("✅ Strong rules found:", len(strong_rules))
print(strong_rules[["antecedents","consequents","support","confidence","lift"]].head(10))


In [ ]:
def recommend(tree, rules_df=strong_rules, top_n=5):
    recs = rules_df[rules_df['antecedents'].apply(lambda x: tree in x)]
    if recs.empty:
        print(f"No recommendations for {tree}")
        return
    recs = recs.sort_values(by=["confidence","lift"], ascending=False)
    for _, r in recs.head(top_n).iterrows():
        print(f"💡 If buyer gets {list(r['antecedents'])} → recommend {list(r['consequents'])} "
              f"(conf={r['confidence']:.2f}, lift={r['lift']:.2f})")

# for instance
recommend("crabapple")
